# Semantic search on telegram channels

Showcasing how to extract telegram messages for a specific channel and perform semantic search given an user query. 
I use pretrained models for semantic search from huggingface.

The fetched messages are first embbeded with the model and then uploaded to a pinecode index. Then the query is embedded and compared against the indexed data.

An option for future work is to fine tune a model specific for telegram messages of certain topic. 

In [2]:
import os
import json
import sys
import pandas as pd
import numpy as np
import openai
import time
import re
import math
from telethon.sync import TelegramClient
from IPython.display import display
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import pinecone

from telethon.tl.types import InputMessagesFilterEmpty

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
pd.options.display.max_columns = None

# do not truncate column width in pandas
pd.options.display.max_colwidth = 200

/Users/camilo.ramirez/Library/Caches/pypoetry/virtualenvs/telegram-nlp-6m6KizWy-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api_id = os.environ["TELEGRAM_API_ID"]
api_hash = os.environ["TELEGRAM_API_HASH"]
pinecone_key = os.environ["PINECONE_APIKEY"]
phone = "+34634454832"
username = "@elvesipeto"
messages = []

## Data fetching and cleaning

In [3]:
pd_data = []

columns = ["channel_name", "id", "peer_id", "date", "message", "out", "mentioned",
        "media_unread", "silent", "post", "from_scheduled", "legacy", 
        "edit_hide", "pinned","noforwards", "from_id", "fwd_from", "via_bot_id",
        "reply_to", "media", "reply_markup", "entities", "views",
        "forwards", "replies", "edit_date", "post_author", "grouped_id",
        "reactions", "restriction_reason", "ttl_period"]

client = TelegramClient(f"sessions_data/{phone}", api_id, api_hash)
channel_id = "@runonflux"
n = 5000

async with client:        
    async for msg in client.iter_messages(channel_id, filter=InputMessagesFilterEmpty, limit=n):
        pd_data.append((channel_id, msg.id,msg.peer_id, msg.date, msg.message,
                msg.out, msg.mentioned, msg.media_unread, msg.silent,msg.post,
                msg.from_scheduled, msg.legacy, msg.edit_hide, msg.pinned, msg.noforwards,
                msg.from_id, msg.fwd_from, msg.via_bot_id, msg.reply_to, msg.media, msg.reply_markup,
                msg.entities, msg.views, msg.forwards, msg.replies, msg.edit_date, msg.post_author,
                msg.grouped_id, msg.reactions, msg.restriction_reason, msg.ttl_period
        ))

df = pd.DataFrame(pd_data, columns=columns)

In [7]:
# Data cleaning and filtering 

df["token_count"] = df["message"].apply(lambda x: len(x.split(" "))  if type(x) == str else 0)
print("Total message count", len(df))

filtered_df = df[~df["message"].isna()]
print("Total messages after excluding empty", len(filtered_df))

# What is the distribution of token counts?
display(filtered_df["token_count"].quantile([.1, .25, .5, .75, .95, 0.98]))


lower_limit = 4

# 250 was the max input lenght of the training data for multi-qa-MiniLM-L6-cos-v1
upper_limit = 250 

filtered_df = filtered_df[(filtered_df["token_count"] > lower_limit) & (filtered_df["token_count"] < upper_limit)]
print("Total messages after excluding long and short messages", len(filtered_df))

# clean messages
filtered_df.loc[:, "clean_message"] = filtered_df["message"].apply(lambda x: re.sub('[^A-Za-z0-9 .,?$%\'"]+', '', x))

# remove duplicates
filtered_df = filtered_df[~filtered_df.duplicated(subset=["clean_message"])]
print("Total messages after excluding duplicates", len(filtered_df))

Total message count 5000
Total messages after excluding empty 4848


0.10      1.00
0.25      4.00
0.50      9.00
0.75     19.00
0.95     56.00
0.98    132.06
Name: token_count, dtype: float64

Total messages after excluding long and short messages 3459
Total messages after excluding duplicates 3117


In [9]:
filtered_df.to_csv(f"notebooks/data/{channel_id}.csv", index=False)

## Encoding with Pytorch

In [11]:
%load_ext autoreload
%autoreload 2
from semantic_search_generator import SemanticSearchGenerator

In [27]:
channel_id = "@runonflux"
df = pd.read_csv(f"notebooks/data/{channel_id}.csv")
messages = df.sample(10, random_state=11)["clean_message"].to_list()
messages

['This flux 4 upgrade made 6 of my nodes go offline.  Now is dos mode.How to get out if this please?',
 'I would suggest you to stop promoting the group BettyK0',
 'Yes deleting nodes because of low rewards is the most scary thing  we may even see less than 0.20 ',
 'Join our CBO Davy Wittock for a special mining episode for Around the Blockchain today at 5 PM EST Get ready for an exciting discussion about Proof of Useful Work and the future of mining Link  httpsyoutube.comaroundtheblockchainofficial',
 ' many crypto projects are one sided in their community. Most are in discord, some are almost exclusively on telegram, some loosely on Twitter....',
 "My buy in was 60c but I asked literally everyone big in the flux space if we could maintain the dollar and they were all bullish including Dan. I wanted to sell to buy back cheaper and am kicking myself I didn't lol",
 "What's command on TG Flux  to check amount stake titan node ?",
 'For example, my stratus gives me 110 a month with curr

In [31]:
generator = SemanticSearchGenerator()

# Text to encode
query = "investment strategy"
doc_score_pairs = generator.search_batch(query, messages, device="mps")

#Output passages & scores
for doc, score in doc_score_pairs[:10]:
    print(score, doc)


0.2212686538696289 My buy in was 60c but I asked literally everyone big in the flux space if we could maintain the dollar and they were all bullish including Dan. I wanted to sell to buy back cheaper and am kicking myself I didn't lol
0.10311347246170044 For example, my stratus gives me 110 a month with current price and APR. Cost of the dedicated server is 70.If i wanted to run cumulus instead, it would be 40 cumulus, giving me about 220 total a month, but costing me about 210 in VPS cost.Plus now i have to monitor and maintain 40 nodes instead of one 
0.09914617985486984 Join our CBO Davy Wittock for a special mining episode for Around the Blockchain today at 5 PM EST Get ready for an exciting discussion about Proof of Useful Work and the future of mining Link  httpsyoutube.comaroundtheblockchainofficial
0.08931327611207962 I would suggest you to stop promoting the group BettyK0
0.06712281703948975 Anybody withdraw Flux  from CoinEX?Looks suspended
0.04982435703277588  many crypto pr

## Data indexing

In [5]:
channel_id = "@runonflux"
model_name = "multi-qa-MiniLM-L6-cos-v1"
df = pd.read_csv(f"notebooks/data/{channel_id}.csv")

In [6]:
# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key=os.environ["PINECONE_APIKEY"],
    environment="us-west1-gcp"
)

In [7]:
# embeddings model
multi_qa_encoder = SentenceTransformer(model_name)

query = "This coin will moon soon"
vec = multi_qa_encoder.encode(query, convert_to_tensor=False)

In [8]:
FORCE_DELETE_INDEX = False # Set True only for initializing the index
INDEX_NAME = "telegram-embeddings"

if FORCE_DELETE_INDEX:
    pinecone.delete_index(INDEX_NAME)

if INDEX_NAME not in pinecone.list_indexes():
    pinecone.create_index(INDEX_NAME, dimension=len(vec))
    
# connect to index
index = pinecone.Index(INDEX_NAME)

In [13]:
import torch

def get_device():
    has_gpu = torch.cuda.is_available()
    has_mps = torch.backends.mps.is_built()
    device = "mps" if has_mps else "gpu" if has_gpu else "cpu"
    return device

get_device()

'mps'

In [9]:
# Upload data to index
COMPUTE_EMBEDDINGS = True
if COMPUTE_EMBEDDINGS:
    # create embeddings
    df["embeddings_cpu"] = df["clean_message"].apply(lambda x: multi_qa_encoder.encode(x, device="cpu", convert_to_tensor=True, show_progress_bar=False))  


In [12]:
generator = SemanticSearchGenerator(model_name)

# Faster with mps
df["embeddings_mps"] = df["clean_message"].apply(lambda x: generator.encode_messages(x, device="mps"))

In [13]:
df[["embeddings_cpu", "embeddings_mps"]].head(3)

,embeddings_cpu,embeddings_mps
0,"[tensor(-0.0749), tensor(-0.1107), tensor(-0.0339), tensor(0.0374), tensor(-0.0197), tensor(0.0569), tensor(0.0786), tensor(-0.0522), tensor(-0.0159), tensor(0.0207), tensor(-0.0155), tensor(-0.01...","[[tensor(-0.0749, device='mps:0'), tensor(-0.1107, device='mps:0'), tensor(-0.0339, device='mps:0'), tensor(0.0374, device='mps:0'), tensor(-0.0197, device='mps:0'), tensor(0.0569, device='mps:0')..."
1,"[tensor(0.0309), tensor(-0.0022), tensor(0.0416), tensor(0.0120), tensor(0.0061), tensor(0.0405), tensor(-0.1566), tensor(-0.0018), tensor(-0.1030), tensor(0.0587), tensor(0.0292), tensor(0.0281),...","[[tensor(0.0309, device='mps:0'), tensor(-0.0022, device='mps:0'), tensor(0.0416, device='mps:0'), tensor(0.0120, device='mps:0'), tensor(0.0061, device='mps:0'), tensor(0.0405, device='mps:0'), t..."
2,"[tensor(0.0337), tensor(-0.0346), tensor(-0.0315), tensor(-0.0759), tensor(-0.0189), tensor(0.0352), tensor(-0.0644), tensor(0.0090), tensor(-0.0890), tensor(-0.0099), tensor(-0.1018), tensor(-0.0...","[[tensor(0.0337, device='mps:0'), tensor(-0.0346, device='mps:0'), tensor(-0.0315, device='mps:0'), tensor(-0.0759, device='mps:0'), tensor(-0.0189, device='mps:0'), tensor(0.0352, device='mps:0')..."


## Upload embeddings

In [14]:
import math
UPLOAD_VECTORS = False # only for index initialization
if UPLOAD_VECTORS:    
    batch_size = 1000
    total_batches = math.ceil(len(df) / batch_size)
    start = 0

    for i in range(total_batches):
        index_upsert = [] # always initialize for each batch        
        end = start + batch_size

        print(f"iterating messages {start}-{end}")
        for j, item in df[start:end].iterrows():
            index_upsert += [
                    (str(j), 
                    item["embeddings_cpu"].tolist(),
                    {
                        "clean_message": item["clean_message"],
                        "channel_name": item["channel_name"],
                        "messagee_id": item["id"]
                    })
            ]
        start = end
        print(f"inserting batch {i}")
        index.upsert(vectors=index_upsert) # can contain maximum 1000 items        

## Search on index

In [28]:
query = "bearish outlook"
query_emb = generator.encode_messages(query)

results = index.query(
  vector=query_emb.tolist(),
  top_k=10,
  include_values=False,
  include_metadata=True
)

for item in results["matches"][:5]:
    print(f"\nscore {item['score']}")
    print(item["metadata"]["clean_message"])


score 0.410763919
Typical bear market... Engagement drops, people hesitate. Meanwhile the team continues to build, POUW just presented a couple of weeks ago

score 0.401366264
keep an eye on announcements

score 0.387041956
Nothing happened, bear markets are a bitch

score 0.362539232
If you interested list in our exchange I can help you

score 0.350785732
Bearmarkets dont last forever. Unless you are in a scam project. I dont see any other thing here than non stopping hardwork. So chill 


In [29]:
query = "bullish news"
query_emb = generator.encode_messages(query)

results = index.query(
  vector=query_emb.tolist(),
  top_k=10,
  include_values=False,
  include_metadata=True
)

for item in results["matches"][:5]:
    print(f"\nscore {item['score']}")
    print(item["metadata"]["clean_message"])


score 0.501404643
Hey here A beautiful week to all, its amazing how when the market is bearing or in halt the vibes in communities change, you can see clearly the vision and the rational unlike in bull where everyone is jumping in without research

score 0.4879556
I've never been a fan of the bullish halving narrative, to many disappointed people who fell for the hype.

score 0.407264948
Who said anything about expecting a bull run ? I said i will cash out my flux when it hits 1$ again

score 0.40503487
Is there a bot's news function or roadmap?

score 0.404052913
I know what you mean but in bull it will be worthed


In [30]:
query = "proof of useful work"
query_emb = generator.encode_messages(query)

results = index.query(
  vector=query_emb.tolist(),
  top_k=10,
  include_values=False,
  include_metadata=True
)

for item in results["matches"][:5]:
    print(f"\nscore {item['score']}")
    print(item["metadata"]["clean_message"])


score 0.420437932
In Q3 of 2023, we plan to launch the proof of concept for Proof of Useful Work.We are excited to partner with Octominer to collaborate on finding optimal builds for PoUW Learn more about the strategic partnership at httpsfluxofficial.medium.comfluxpartnerswithoctominer884353d75854

score 0.409998924
proof of concept is like the first iteration  will it work the way its planned  are there any changes needed  testing the concept is good

score 0.396739244
Discover how Proof of Useful Work solves Web2 cloud infrastructure problems and unlocks new possibilities for AI and more. Check out our latest article httpsfluxofficial.medium.comfluxproofofusefulworkablockchainmilestone6ba11d7d52f5

score 0.380016953
Hi $FLUX communityWhat is Proof of Useful Work PoUW by Run on Flux?Let's read onhttpstwitter.comHouseofChimerastatus1694010279544136181

score 0.371297866
Catch the RunOnFlux Proof of Useful Work Announcement today at CypherPunk2023 at 1015am EST LIVE.Watch LIVE httpsyo